In [2]:
def load_your_data():
    """Load your actual tweet emotion dataset"""
    # Load from CSV file
    df = pd.read_csv('your_tweet_data.csv')
    tweets = df['text'].tolist()
    emotions = df['emotion'].tolist()
    return tweets, emotions


In [4]:
# Tweet Emotion Recognition with TensorFlow - Complete Working Code (FIXED)
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# FIXED: Larger sample dataset (5 samples per class = 30 total)
def create_sample_data():
    """Create sample tweet data with enough samples for stratified split"""
    tweets = [
        # Joy - 5 samples
        "I am so happy today!", "Life is wonderful", "Such joy and happiness", "Amazing day", "Feeling great",
        # Sadness - 5 samples
        "This is extremely sad", "Feeling gloomy and low", "This makes me cry", "So depressing", "Very upset",
        # Anger - 5 samples
        "I am furious about this", "This situation is enraging", "Can't stand it", "So mad", "Angry mood",
        # Surprise - 5 samples
        "Wow, what an amazing surprise!", "Didn't expect that", "What a shock", "Truly surprising", "Incredible!",
        # Fear - 5 samples
        "I feel scared and anxious", "Terrified of what's next", "This is frightening", "So scared", "Really scared",
        # Love - 5 samples
        "I absolutely love this amazing thing", "Such a loving heart", "Love is everywhere", "Pure love", "Feeling loved"
    ]

    emotions = [
        'joy', 'joy', 'joy', 'joy', 'joy',
        'sadness', 'sadness', 'sadness', 'sadness', 'sadness',
        'anger', 'anger', 'anger', 'anger', 'anger',
        'surprise', 'surprise', 'surprise', 'surprise', 'surprise',
        'fear', 'fear', 'fear', 'fear', 'fear',
        'love', 'love', 'love', 'love', 'love'
    ]

    return tweets, emotions

# Data preprocessing function
def preprocess_data(texts, labels, max_features=5000, max_len=50):
    """Preprocess text data and labels"""
    # Tokenize texts
    tokenizer = Tokenizer(num_words=max_features, split=' ')
    tokenizer.fit_on_texts(texts)
    X = tokenizer.texts_to_sequences(texts)
    X = pad_sequences(X, maxlen=max_len)

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(labels)
    y = tf.keras.utils.to_categorical(y_encoded)

    return X, y, tokenizer, label_encoder

# Model creation function
def create_model(max_features=5000, max_len=50, embed_dim=100, lstm_units=64, num_classes=6):
    """Create RNN model for emotion classification"""
    model = Sequential([
        Embedding(max_features, embed_dim, input_length=max_len),
        SpatialDropout1D(0.4),
        LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    return model

# Prediction function
def predict_emotion(model, tokenizer, label_encoder, text, max_len=50):
    """Predict emotion for new text"""
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded, verbose=0)
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)
    emotion = label_encoder.inverse_transform([predicted_class])[0]
    return emotion, confidence

# Main execution
def main():
    print("=== Tweet Emotion Recognition with TensorFlow ===\n")

    # Step 1: Load and prepare data
    print("1. Loading and preprocessing data...")
    tweets, emotions = create_sample_data()
    X, y, tokenizer, label_encoder = preprocess_data(tweets, emotions)

    print(f"   - Number of tweets: {len(tweets)}")
    print(f"   - Number of emotion classes: {len(set(emotions))}")
    print(f"   - Input shape: {X.shape}")
    print(f"   - Output shape: {y.shape}\n")

    # Step 2: Split data (FIXED: Use 25% test size which gives at least 6 test samples)
    print("2. Splitting data...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )
    print(f"   - Training samples: {len(X_train)}")
    print(f"   - Test samples: {len(X_test)}")

    # Step 3: Create model
    print("\n3. Creating model...")
    model = create_model(num_classes=len(set(emotions)))
    model.summary()

    # Step 4: Train model
    print("\n4. Training model...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=20,
        batch_size=4,
        verbose=1
    )

    # Step 5: Evaluate model
    print("\n5. Evaluating model...")
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"   Test Accuracy: {test_accuracy * 100:.2f}%")

    # Step 6: Test predictions
    print("\n6. Testing predictions...")
    test_texts = [
        "I'm extremely happy about this news!",
        "This makes me feel very sad",
        "I'm so angry right now"
    ]

    for text in test_texts:
        emotion, confidence = predict_emotion(model, tokenizer, label_encoder, text)
        print(f"   Text: '{text}'")
        print(f"   Predicted Emotion: {emotion} (Confidence: {confidence:.2f})\n")

    # Save model
    model.save('tweet_emotion_model.h5')
    print("7. Model saved as 'tweet_emotion_model.h5'")

    return model, tokenizer, label_encoder

# Run the program
if __name__ == "__main__":
    model, tokenizer, label_encoder = main()


=== Tweet Emotion Recognition with TensorFlow ===

1. Loading and preprocessing data...
   - Number of tweets: 30
   - Number of emotion classes: 6
   - Input shape: (30, 50)
   - Output shape: (30, 6)

2. Splitting data...
   - Training samples: 22
   - Test samples: 8

3. Creating model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


4. Training model...
Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 183ms/step - accuracy: 0.0700 - loss: 1.8006 - val_accuracy: 0.0000e+00 - val_loss: 1.7941
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.2966 - loss: 1.7840 - val_accuracy: 0.0000e+00 - val_loss: 1.7997
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.3748 - loss: 1.7710 - val_accuracy: 0.1250 - val_loss: 1.8062
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.3218 - loss: 1.7671 - val_accuracy: 0.0000e+00 - val_loss: 1.8105
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.3752 - loss: 1.7690 - val_accuracy: 0.1250 - val_loss: 1.8138
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2838 - loss: 1.7550 - val_accuracy: 0.1250 - val_loss: 1.8202
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.3860 - loss: 1.7363 - val_accuracy: 0.1250 - val_loss: 1.8245
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 0.4585 - loss: 1.7111 - val_a

   Text: 'I'm so angry right now'
   Predicted Emotion: anger (Confidence: 0.25)

7. Model saved as 'tweet_emotion_model.h5'
